## Step 1: Install Packages

In [ ]:
%%capture
!pip install - huggingface_hub
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

## Step 3: import Required libraries

In [ ]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
os.environ["HF_Token"] = "hf_"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_"

In [ ]:
# this is basically the system prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output)
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Meta-Llama-3-8b-Instruct"

# The instruction dataset to use
dataset_name = "/content"

# Fine-tuned model name
new_model = "Llama-3-8b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}


## Step 4: load everything and start fine-tunning process

In [ ]:
# Load dataset (you can process it here)
dataset = load_dataset("json",split = "train", data_files="/content/my_file.json")
dataset = dataset.map(formatting_prompts_func, batched = True,)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

Dataset({
    features: ['input', 'instruction', 'output', 'text'],
    num_rows: 53
})


In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# # Check GPU compatibility with bfloat16
# if compute_dtype == torch.float16 and use_4bit:
#     major, _ = torch.cuda.get_device_capability()
#     if major >= 8:
#         print("=" * 80)
#         print("Your GPU supports bfloat16: accelerate training with bf16=True")
#         print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8b-Instruct and are newly initialized: ['model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.31.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.24.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.25.self_at

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/53 [00:00<?, ? examples/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=14, training_loss=1.8914604187011719, metrics={'train_runtime': 44.8328, 'train_samples_per_second': 1.182, 'train_steps_per_second': 0.312, 'total_flos': 282014749040640.0, 'train_loss': 1.8914604187011719, 'epoch': 1.0})

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = alpaca_prompt.format(
          "parse the HTMLblock in input to identify and extract meaningful attributes relevant to e-commerce contexts (e.g., product names, prices, descriptions, images) and also identify the CSS selectors from class of html to pinpoint their location like top, bottom, left, right. Response should be in like example product:iPhone 12,description:The latest iPhone model with A14 Bionic chip,price:$799,image:iphone12.jpg,location:top",
          "<body class='top'><h1>Product: Robotic Vacuum Cleaner</h1><p>Description: Smart robotic vacuum cleaner with mapping technology.</p><p>Price: $299</p><img src='robotic-vacuum.jpg' alt='Robotic Vacuum Cleaner'></body>",
          "", )

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "",# instruction
        "<div class='middle'><h1>Product: HDMI Cable (6ft)</h1><p>Description: High-speed HDMI cable for 4K video and audio transmission.</p><p>Price: $8</p><img src='hdmi-cable.jpg' alt='HDMI Cable (6ft)'></div>",# input
        "" # output
    )
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nparse the HTMLblock in input to identify and extract meaningful attributes relevant to e-commerce contexts (e.g., product names, prices, descriptions, images) and also identify the CSS selectors from class of html to pinpoint their location like top, bottom, left, right. Response should be in like example product:iPhone 12,description:The latest iPhone model with A14 Bionic chip,price:$799,image:iphone12.jpg,location:top\n\n### Input:\n<div class='middle'><h1>Product: HDMI Cable (6ft)</h1><p>Description: High-speed HDMI cable for 4K video and audio transmission.</p><p>Price: $8</p><img src='hdmi-cable.jpg' alt='HDMI Cable (6ft)'></div>\n\n### Response:\nproduct:HDMI Cable (6ft),description:High-speed HDMI cable for 4K video and audio transmission.,price:$8,image:hdmi-cable.jpg,location:middl

In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "parse the HTMLblock in input to identify and extract meaningful attributes relevant to e-commerce contexts (e.g., product names, prices, descriptions, images) and also identify the CSS selectors from class of html to pinpoint their location like top, bottom, left, right. Response should be in like example product:iPhone 12,description:The latest iPhone model with A14 Bionic chip,price:$799,image:iphone12.jpg,location:top",# instruction
        "<div class='right'><h1>Product: Laptop Backpack</h1><p>Description: Stylish laptop backpack with compartments for tech accessories.</p><p>Price: $35</p><img src='laptop-backpack.jpg' alt='Laptop Backpack'></div>",# input
        "" # output
    )
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nparse the HTMLblock in input to identify and extract meaningful attributes relevant to e-commerce contexts (e.g., product names, prices, descriptions, images) and also identify the CSS selectors from class of html to pinpoint their location like top, bottom, left, right. Response should be in like example product:iPhone 12,description:The latest iPhone model with A14 Bionic chip,price:$799,image:iphone12.jpg,location:top\n\n### Input:\n<div class='right'><h1>Product: Laptop Backpack</h1><p>Description: Stylish laptop backpack with compartments for tech accessories.</p><p>Price: $35</p><img src='laptop-backpack.jpg' alt='Laptop Backpack'></div>\n\n### Response:\nproduct:Laptop Backpack,description:Stylish laptop backpack with compartments for tech accessories.,price:$35,image:laptop-backpack.

In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "parse the HTMLblock in input to identify and extract meaningful attributes relevant to e-commerce contexts (e.g., product names, prices, descriptions, images) and also identify the CSS selectors from class of html to pinpoint their location like top, bottom, left, right. Response should be in like example product:iPhone 12,description:The latest iPhone model with A14 Bionic chip,price:$799,image:iphone12.jpg,location:top",# instruction
        "<div class='left'><h1>Product: Power Strip with USB</h1><p>Description: Power strip with USB ports for charging multiple devices.</p><p>Price: $20</p><img src='power-strip-usb.jpg' alt='Power Strip with USB'></div>",# input
        "" # output
    )
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nparse the HTMLblock in input to identify and extract meaningful attributes relevant to e-commerce contexts (e.g., product names, prices, descriptions, images) and also identify the CSS selectors from class of html to pinpoint their location like top, bottom, left, right. Response should be in like example product:iPhone 12,description:The latest iPhone model with A14 Bionic chip,price:$799,image:iphone12.jpg,location:top\n\n### Input:\n<div class='left'><h1>Product: Power Strip with USB</h1><p>Description: Power strip with USB ports for charging multiple devices.</p><p>Price: $20</p><img src='power-strip-usb.jpg' alt='Power Strip with USB'></div>\n\n### Response:\nproduct:Power Strip with USB,description:Power strip with USB ports for charging multiple devices.,price:$20,image:power-strip-us

In [ ]:
# Taking the response part from generated text and change it into dictionary, so it can be jsonify later on for api
def generated_text_to_dic(generated_text):
    x = generated_text[0].split('### Response:\n')

    y = x[1].split('\n\n')

    z = y[0].split(',')

    dic = {}

    for i in z:
        j = i.split(":")
        dic[j[0]] = j[1]

    return dic